# Objectives
- 노트북 가격 예측(회귀)

## Functions

In [1]:
import numpy as np
import pandas as pd
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score, TimeSeriesSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import linear_kernel
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, VotingRegressor, StackingRegressor, AdaBoostRegressor
from lightgbm import LGBMClassifier
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf
from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import re
import nltk 
from nltk.corpus import stopwords
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
from sentence_transformers import SentenceTransformer

## Load Data

In [2]:
file_path = 'D:\Code\Datasets/laptop_data_cleaned.csv'

df = pd.read_csv(file_path)

print(f'데이터프레임 원본 : \n{df}')

데이터프레임 원본 : 
     Company            TypeName  Ram  Weight      Price  TouchScreen  Ips  \
0      Apple           Ultrabook    8    1.37  11.175755            0    1   
1      Apple           Ultrabook    8    1.34  10.776777            0    0   
2         HP            Notebook    8    1.86  10.329931            0    0   
3      Apple           Ultrabook   16    1.83  11.814476            0    1   
4      Apple           Ultrabook    8    1.37  11.473101            0    1   
...      ...                 ...  ...     ...        ...          ...  ...   
1268    Asus            Notebook    4    2.20  10.555257            0    0   
1269  Lenovo  2 in 1 Convertible    4    1.80  10.433899            1    1   
1270  Lenovo  2 in 1 Convertible   16    1.30  11.288115            1    1   
1271  Lenovo            Notebook    2    1.50   9.409283            0    0   
1272      HP            Notebook    6    2.19  10.614129            0    0   

             Ppi              Cpu_brand   HDD  SSD

## Data Pre-Processing

In [3]:
# 데이터의 전반적인 정보 확인 #
'''

컬럼 정보
Company : 회사
TypeName : 노트북 종류
Ram : 메모리
Weight : 무게
Price : 가격
TouchScreen : 터치 스크린
IPS : IPS 화면
Ppi : 해상도의 밀도 단위
Cpu_brand : CPU 사양
HDD : hdd 용량
SSD : ssd 용량
Gpu_brand : 그래픽카드 브랜드
Os : 운영체제

'''

'\n\n컬럼 정보\nCompany : 회사\nTypeName : 노트북 종류\nRam : 메모리\nWeight : 무게\nPrice : 가격\nTouchScreen : 터치 스크린\nIPS : IPS 화면\nPpi : 해상도의 밀도 단위\nCpu_brand : CPU 사양\nHDD : hdd 용량\nSSD : ssd 용량\nGpu_brand : 그래픽카드 브랜드\nOs : 운영체제\n\n'

In [4]:
# 누락 데이터 확인

df.isnull().sum()

Company        0
TypeName       0
Ram            0
Weight         0
Price          0
TouchScreen    0
Ips            0
Ppi            0
Cpu_brand      0
HDD            0
SSD            0
Gpu_brand      0
Os             0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1273 entries, 0 to 1272
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1273 non-null   object 
 1   TypeName     1273 non-null   object 
 2   Ram          1273 non-null   int64  
 3   Weight       1273 non-null   float64
 4   Price        1273 non-null   float64
 5   TouchScreen  1273 non-null   int64  
 6   Ips          1273 non-null   int64  
 7   Ppi          1273 non-null   float64
 8   Cpu_brand    1273 non-null   object 
 9   HDD          1273 non-null   int64  
 10  SSD          1273 non-null   int64  
 11  Gpu_brand    1273 non-null   object 
 12  Os           1273 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 129.4+ KB


### 데이터 분석 목적에 영향이 없는 컬럼 제거

In [6]:
df.drop(columns = ['Company'], inplace = True)

print(df)

                TypeName  Ram  Weight      Price  TouchScreen  Ips  \
0              Ultrabook    8    1.37  11.175755            0    1   
1              Ultrabook    8    1.34  10.776777            0    0   
2               Notebook    8    1.86  10.329931            0    0   
3              Ultrabook   16    1.83  11.814476            0    1   
4              Ultrabook    8    1.37  11.473101            0    1   
...                  ...  ...     ...        ...          ...  ...   
1268            Notebook    4    2.20  10.555257            0    0   
1269  2 in 1 Convertible    4    1.80  10.433899            1    1   
1270  2 in 1 Convertible   16    1.30  11.288115            1    1   
1271            Notebook    2    1.50   9.409283            0    0   
1272            Notebook    6    2.19  10.614129            0    0   

             Ppi              Cpu_brand   HDD  SSD Gpu_brand       Os  
0     226.983005          Intel Core i5     0  128     Intel      Mac  
1     127.67794

In [7]:
df.drop(columns = ['TypeName'], inplace=True)

In [8]:
print(df)

      Ram  Weight      Price  TouchScreen  Ips         Ppi  \
0       8    1.37  11.175755            0    1  226.983005   
1       8    1.34  10.776777            0    0  127.677940   
2       8    1.86  10.329931            0    0  141.211998   
3      16    1.83  11.814476            0    1  220.534624   
4       8    1.37  11.473101            0    1  226.983005   
...   ...     ...        ...          ...  ...         ...   
1268    4    2.20  10.555257            0    0  100.454670   
1269    4    1.80  10.433899            1    1  157.350512   
1270   16    1.30  11.288115            1    1  276.053530   
1271    2    1.50   9.409283            0    0  111.935204   
1272    6    2.19  10.614129            0    0  100.454670   

                  Cpu_brand   HDD  SSD Gpu_brand       Os  
0             Intel Core i5     0  128     Intel      Mac  
1             Intel Core i5     0    0     Intel      Mac  
2             Intel Core i5     0  256     Intel   Others  
3             I